In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# IMPORTS

In [ ]:
import seaborn as sns
import nltk
nltk.download()

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# **1)Lets understand the dataset**

In [ ]:
filename = '/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv'
df = pd.read_csv(filename)
df.head()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
sns.countplot(x = df['sentiment'])

# Dataset Properties
* It has 2 columns Review and sentiment.
* Review has textual data and will be the input to the model.
* There are 50,000 reviews in the dataset
* Sentiment has two values positive or negative and will be the output to the model.
* There are 25,000 positive and negative reviews each.
* Dataset has a uniform distribution.

# Data Preprocessing

In [ ]:
df['review'][0]

*After looking at the text , following tasks to be performed*
* Everything in lower case
* Removing punctuation marks.
* Removing all stopwords
* Removing break statements
* Using *PorterStemmer* to convert words back to their root stems.
I'll first test above on 1 string alone and then loop it over.

In [ ]:
ps = PorterStemmer()
filter_review = []
for sentence in df['review']:
    filter_sentence = []
    sentence = sentence .replace('<br /><br />'," ")
    sentence = re.sub('[^a-zA-Z]',' ',sentence)
    sentence = sentence.lower()
    for word in nltk.word_tokenize(sentence) :
        if word not in stopwords.words('english'):
            filter_sentence.append(ps.stem(word))
    filter_sentence = ' '.join(word for word in filter_sentence)
    filter_review.append(filter_sentence)

In [ ]:
#Lets view the transformed text
filter_review[1:3]

Now we are done with the text pre processing and we will now proceed for vectorization

**Now that we have completed Text preprocessing , we need to perform vectorization as we can't apply models on textual data.
* We will use TFIDF for vectorization of review column**
* Convert sentiment column 

In [ ]:
tf = TfidfVectorizer(max_features = 10000)
X = tf.fit_transform(filter_review).todense()


*Since our output is also in text form lets convert it into numerical form*

In [ ]:
df['sentiment'] = df['sentiment'].apply(lambda x : 0 if x=='positive' else 1)

# Creating Test and Training Data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,df['sentiment'],test_size = 0.3,random_state = 90)

# Applying Model

In [ ]:
#Logistic Regression
lm = LogisticRegression()
lm.fit(X_train,Y_train)
lm_score = lm.score(X_test,Y_test)
#RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)
rf_score = rf.score(X_test,Y_test)
print('LogisticRegression: '+ str(lm_score) +'\nRandom Forest: '+ str(rf_score))
